In [1]:
import sklearn
import gensim
import pandas as pd 
import numpy as np
import nltk
import logging
from nltk.corpus import stopwords
from nltk import word_tokenize
import string

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model.logistic import LogisticRegression
from sklearn.model_selection import train_test_split, cross_val_score

from sklearn import preprocessing

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

dowload NLTK

In [2]:
#nltk.download('all')

Load our dataset using pandas

In [3]:
train_df = pd.read_table("C:\\Users\\chris_000\\Desktop\\train2017.tsv",names=["Id1", "Id2", "Sentiment", "Tweet"],engine="python", sep="\t+",escapechar="\\")

In [4]:
train_df= train_df[['Sentiment','Tweet']]
train_df.head()

,Sentiment,Tweet
0,positive,Gas by my house hit $3.39!!!! I'm going to Cha...
1,negative,"Theo Walcott is still shit, watch Rafa and Joh..."
2,negative,"its not that I'm a GSP fan, i just hate Nick D..."
3,negative,Iranian general says Israel's Iron Dome can't ...
4,neutral,"Tehran, Mon Amour: Obama Tried to Establish Ti..."


Create 3 seperate datasets pos/neg, neg/neut, pos/neut for our 3 classifiers


In [5]:
import io

train_df_1 = train_df[train_df['Sentiment']!='negative']

train_df_2 = train_df[train_df['Sentiment']!='positive']

train_df_3 = train_df[train_df['Sentiment']!='neutral']

#create numeric categories using the LabelEncoder and fit-transorm pipeline
le = preprocessing.LabelEncoder()

le.fit(train_df['Sentiment'])
label_train = le.transform(train_df['Sentiment'])
label_train_1 = le.transform(train_df_1['Sentiment'])
label_train_2 = le.transform(train_df_2['Sentiment'])
label_train_3  = le.transform(train_df_3['Sentiment'])

print(label_train)


base = 'C:\\Users\\chris_000\\Desktop\\'
f = io.open(base+'SemEval2017_task4_subtaskA_test_english_gold.txt', "r", encoding="utf-8")
sentiments = []
for i, line in enumerate(f):
    values = line.split()
    #print values
    sentiments.append(values[1])
test_df = pd.read_table("C:\\Users\\chris_000\\Desktop\\test2017.tsv",names=["Id1", "Id2", "Sentiment", "Tweet"],engine="python", sep="\t+",escapechar="\\")


test_df=test_df.assign(Label=sentiments)

msg_test, msg ,label_test_final,lbl = train_test_split(test_df['Tweet'], test_df['Label'],test_size = 0,shuffle = False)

#create numeric categories using the LabelEncoder and fit-transorm pipeline
label_test_final = le.transform(label_test_final)
print(label_test_final)

[2 0 0 ... 1 2 1]
[1 2 2 ... 1 1 1]


In [13]:
X_train_2

<16544x33638 sparse matrix of type '<type 'numpy.float64'>'
	with 287315 stored elements in Compressed Sparse Row format>

In [14]:

vectorizer_2 = TfidfVectorizer()
X_train = vectorizer_2.fit_transform(train_df['Tweet'])
X_train_1 = vectorizer_2.transform(train_df_1['Tweet'])
X_train_2 = vectorizer_2.transform(train_df_2['Tweet'])
X_train_3 = vectorizer_2.transform(train_df_3['Tweet'])

X_test = vectorizer_2.transform(msg_test)

Use linear svm for our binary classification as it's both good-performing and cost-effective

In [16]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import precision_score, f1_score
from sklearn.model_selection import cross_validate
from sklearn import metrics
from sklearn.svm import SVC, LinearSVC




svc = LinearSVC(C=0.8)
classifier = svc

classifier.fit(X_train_1, label_train_1)
test_1 = classifier.predict(X_test)
train_1 = classifier.predict(X_train)
#print "tfidf",n,metrics.accuracy_score(label_test, predictions)

classifier.fit(X_train_2, label_train_2)
test_2 = classifier.predict(X_test)
train_2 = classifier.predict(X_train)

classifier.fit(X_train_3, label_train_3)
test_3 = classifier.predict(X_test)
train_3 = classifier.predict(X_train)
    

In [17]:
rr_test=np.stack((test_1,test_2,test_3), axis=1)
rr_train=np.stack((train_1,train_2,train_3), axis=1)

In [18]:
knn = KNeighborsClassifier(n_neighbors=5)

classifier = knn
classifier.fit(rr_train, label_train)
predictions = classifier.predict(rr_test)
acc = metrics.accuracy_score(label_test_final, predictions)
f1 = (f1_score(label_test_final, predictions, average='macro')) 

0.5524377522219237
0.5681374145229567


In [23]:
print("F1:",f1)
print("Accuracy:",acc)

('F1:', 0.5524377522219237)
('Accuracy:', 0.5681374145229567)


Both metrics are better than everything but the rbf svm which performs better at accuracy.In general, stackings techniques and other meta-classifiers tend to perform better than simple ones and even beat state-of-the-art classifiers using simple structures as the main components (e.g. random forests classifier)